Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 019
Code Title: | Emerging Technologies in CpE 2
2nd Semester | AY 2023-2024
<hr> | <hr>
<u>**Final Exam** | Model Deployment in the Cloud
**Name** | Abad, Julia Marie Iberet
            | Leopando, Lara Jeanene   
**Section** | CPE32S3
**Date Performed**: | May 14, 2024
**Date Submitted**: | May 18, 2024
**Instructor**: | Engr. Roman Richard



#Objective(s):

This activity aims to apply all the learnings for the Final Period.



#Intended Learning Outcomes (ILOs):

- Demonstrate how to train and save a model.
- Demonstrate how to deploy the deep learning model in the cloud. (not Machine Learning model)


#Instructions:

- You can choose any previous deep learning model.
- Follow the instructions on deploying a model using Streamlit App in the cloud.

#About the Dataset

The Dataset we used: **Cats and Dogs image classification** <br><br>
**Link of the Dataset:** https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification <br><br>

- Contains mages of cats and dogs scraped off of google images. The problem statement is to build a model that can classify between a cat and a dog in an image as accurately as possible.<br>

- Image sizes range from roughly 100x100 pixels to 2000x1000 pixels.<br>

- Image format is jpeg.

#Problem that the dataset is solving

The primary problem this project addresses is the automatic classification of images into specific categories—in this case, identifying whether an image contains a cat or a dog. This type of classification is useful in numerous real-world applications. <br><br>

Examples:<br><br>

- Photo Organization: Automatically tagging and organizing large collections of pet photos.
- Animal Shelter Management: Assisting in the automatic categorization of pets in shelters for faster processing and management.
- Content Filtering: Enabling content filtering on social media platforms to manage and sort user-uploaded pet images.
- Pet Recognition: Assisting in pet recognition systems for lost and found services

#Activity Proper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit

In [ ]:
!pip install gdown

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat'

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = datagen.flow_from_directory(
    os.path.join(path, 'train'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    os.path.join(path, 'train'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 447 images belonging to 2 classes.
Found 110 images belonging to 2 classes.


##Training the model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

Epoch 1/20
13/13 [==============================] - 112s 8s/step - loss: 0.7204 - accuracy: 0.5181 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/20
13/13 [==============================] - 86s 7s/step - loss: 0.6933 - accuracy: 0.5301 - val_loss: 0.6987 - val_accuracy: 0.4792
Epoch 3/20
13/13 [==============================] - 86s 6s/step - loss: 0.6906 - accuracy: 0.5952 - val_loss: 0.6853 - val_accuracy: 0.5000
Epoch 4/20
13/13 [==============================] - 85s 6s/step - loss: 0.6750 - accuracy: 0.5663 - val_loss: 0.7027 - val_accuracy: 0.5521
Epoch 5/20
13/13 [==============================] - 92s 7s/step - loss: 0.6645 - accuracy: 0.6024 - val_loss: 0.6752 - val_accuracy: 0.5938
Epoch 6/20
13/13 [==============================] - 82s 6s/step - loss: 0.6485 - accuracy: 0.6048 - val_loss: 0.7001 - val_accuracy: 0.5625
Epoch 7/20
13/13 [==============================] - 81s 6s/step - loss: 0.6275 - accuracy: 0.6651 - val_loss: 0.6504 - val_accuracy: 0.5625
Epoch 8/20
13/13 [=

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/cat_dog_classifier.h5')

##Streamlit

In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image
import gdown
import os

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/cat_dog_classifier.h5')

In [ ]:
class_names = ['cat', 'dog']

In [ ]:
def predict(image):
    image = image.resize((256, 256))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    st.write(f"Raw model output: {prediction}")
    predicted_class = class_names[int(np.round(prediction[0]))]
    return predicted_class

st.title("Cat and Dog Image Classification App")
uploaded_file = st.file_uploader("Please choose an image:  ", type="jpg")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    if model:
        label = predict(image)
        st.write(f"Prediction: {label}")
    else:
        st.error("Model is not loaded, unable to classify the image.")

##Creating app.py and requirements.txt for deployment

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/app.py'

In [ ]:
script_code = """
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image
import gdown

model_url = 'https://drive.google.com/file/d/1-IQghpVqloccTnJJVD8sJiI3kuYsaz-8'
model_path = 'cat_dog_classifier.h5'

@st.cache(allow_output_mutation=True)
def load_model():
    gdown.download(model_url, model_path, quiet=False)
    model = tf.keras.models.load_model(model_path)
    return model

model = load_model()

class_names = ['cat', 'dog']

def predict(image):
    image = image.resize((256, 256))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    predicted_class = class_names[int(np.round(prediction[0]))]
    return predicted_class

st.title("Cat and Dog Image Classification App")
uploaded_file = st.file_uploader("Please choose an image:  ", type="jpg")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    label = predict(image)
    st.write(f"Prediction: {label}")
"""

with open(save_path, 'w') as f:
    f.write(script_code)

print(f'Script saved to Google Drive at: {save_path}')

Script saved to Google Drive at: /content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/app.py


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/requirements.txt'
requirements_content = """
streamlit
tensorflow
pillow
numpy
gdown
"""

file = drive.CreateFile({'title': 'requirements.txt', 'parents': [{'id': '1g-0Wwo7bNZJJNQYM9uFKDBsBgqN7CsaS'}]})
file.SetContentString(requirements_content)
file.Upload()
print(f"File saved to Google Drive at: {save_path}")

File saved to Google Drive at: /content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/requirements.txt


In [ ]:
import os

model_path = '/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/cat_dog_classifier.h5'

print(f"Model file path: {os.path.abspath(model_path)}")
print(f"Model file size: {os.path.getsize(model_path)} bytes")

try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")


Model file path: /content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/DogCat/cat_dog_classifier.h5
Model file size: 158685896 bytes
Model loaded successfully.
